# Get Additional Metadata Features
This script looks into the additional table options available on the cluster. Ultimately, we didn't use these as the additional tables were missing too many tracks

In [1]:
import pandas as pd
import numpy as np
import sys
import os
import sqlite3

In [3]:
#set file paths for dbs
data_dir = '/scratch/work/courses/DSGA1004-2021/MSD/AdditionalFiles/'
os.chdir('/home/jte2004/final-project-team_wildfire/')

test = os.path.join(data_dir, 'unique_terms.txt')
lastfm = os.path.join(data_dir, 'lastfm_tags.db')
mxm = os.path.join(data_dir, 'mxm_dataset.db') 
print(os.getcwd())

/home/jte2004/final-project-team_wildfire


In [2]:
#last fm missing songs

In [4]:
tables = """SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%'"""

In [5]:
# Read the songs database table into a dataframe -- with selected columns
con = sqlite3.connect(lastfm)
con = sqlite3.connect(mxm)
tab_list = pd.read_sql_query(tables, con)
con.close()


print(tab_list.shape)
tab_list.head()

(2, 1)


,name
0,words
1,lyrics


In [7]:
# Read the songs database table into a dataframe -- with selected columns
con = sqlite3.connect(lastfm)
tid_tag_df = pd.read_sql_query("""SELECT 
                            *
                        FROM 
                            tid_tag""", con)
con.close()


print(tid_tag_df.shape)
tid_tag_df.head()

(8598630, 3)


,tid,tag,val
0,1,1,100.0
1,1,2,100.0
2,2,3,100.0
3,2,4,100.0
4,3,5,100.0


In [8]:
# Read the songs database table into a dataframe -- with selected columns
con = sqlite3.connect(lastfm)
tag_df = pd.read_sql_query("""SELECT 
                            *
                        FROM 
                            tags""", con)
con.close()


print(tag_df.shape)
tag_df.head()

(522366, 1)


,tag
0,classic rock
1,Progressive rock
2,blues
3,memphis slim
4,pop


In [9]:
# Read the songs database table into a dataframe -- with selected columns
con = sqlite3.connect(lastfm)
tid_df = pd.read_sql_query("""SELECT 
                            *
                        FROM 
                            tids""", con)
con.close()

print(tid_df.shape)
tid_df.head()

(505216, 1)


,tid
0,TRCCCYE12903CFF0E9
1,TRCCCFH12903CEBC70
2,TRCCCJT128F429FFF6
3,TRCCCBJ128F4286E6F
4,TRCCJTI128EF35394A


In [10]:
tid_df.nunique()

tid    505216
dtype: int64

In [11]:
#assume the index is the key to join on in the tid_tag table

query = """SELECT * 
        FROM sqlite_master
        LIMIT 10"""

# Read the songs database table into a dataframe -- with selected columns
con = sqlite3.connect(lastfm)
master = pd.read_sql_query(query, con)
con.close()


print(master.shape)
master.head()

(8, 5)


,type,name,tbl_name,rootpage,sql
0,table,tags,tags,2,CREATE TABLE tags (tag TEXT)
1,table,tids,tids,3,CREATE TABLE tids (tid TEXT)
2,table,tid_tag,tid_tag,4,"CREATE TABLE tid_tag (tid INT, tag INT, val FL..."
3,index,idx_tid,tids,41982,CREATE INDEX idx_tid ON tids (tid)
4,index,idx_tag,tags,45769,CREATE INDEX idx_tag ON tags (tag)


In [12]:
first_join = pd.merge(tid_df, tid_tag_df, how='inner', left_index=True, right_on='tid', copy=False)

In [13]:
joined = pd.merge(first_join, tag_df, how='inner', left_on='tag', right_index=True, copy=False)
joined = joined.drop(columns=['tag','tid','tid_y','tag_x'])
joined = joined.rename(columns={"tid_x": "tid", "tag_y": "tag"})
joined.head()

,tid,val,tag
0,TRCCCFH12903CEBC70,100.0,Progressive rock
887,TRCCEGW128F92FE7DE,100.0,Progressive rock
911,TRCCEXX128F4292E09,3.0,Progressive rock
1348,TRCCZHC128F425858F,50.0,Progressive rock
1408,TRCCTIL128F4219729,33.0,Progressive rock


In [14]:
print('shape:', joined.shape)

joined.nunique()

shape: (8598620, 3)


tid    505215
val       101
tag    522365
dtype: int64

In [22]:
# Read the songs database table into a dataframe -- with selected columns
con = sqlite3.connect(mxm)
lyrics_df = pd.read_sql_query("""SELECT 
                            *
                        FROM 
                            lyrics limit 3""", con)
con.close()


print(lyrics_df.shape)
lyrics_df.head()

(3, 5)


,track_id,mxm_tid,word,count,is_test
0,TRAAAAV128F421A322,4623710,i,6,0
1,TRAAAAV128F421A322,4623710,the,4,0
2,TRAAAAV128F421A322,4623710,you,2,0


In [23]:
# Read the songs database table into a dataframe -- with selected columns
con = sqlite3.connect(mxm)
lyrics_df = pd.read_sql_query("""SELECT 
                            track_id,
                            count(distinct(word)) as numwords
                        FROM lyrics
                        GROUP BY track_id
                        limit 10""", con)
con.close()


print(lyrics_df.shape)
lyrics_df.head()

(10, 2)


,track_id,numwords
0,TRAAAAV128F421A322,68
1,TRAAABD128F429CF47,74
2,TRAAAED128E0783FAB,120
3,TRAAAEF128F4273421,59
4,TRAAAEW128F42930C0,63
